In [ ]:
from pathlib import Path

import kagglehub
import keras_cv
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.mobilenet_v2 import (
    preprocess_input as mobilenet_preprocess_input,
)
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    TensorBoard,
)

AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
DATA_DIR = kagglehub.dataset_download("arjuntejaswi/plant-village")
DATA_PATH = Path(DATA_DIR) / "PlantVillage"

print(f"Using dataset from: {DATA_PATH}")

In [ ]:
# Configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42
VALIDATION_SPLIT = 0.2

# Load and split data
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH,
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=True,
    validation_split=VALIDATION_SPLIT,
    subset="training",
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH,
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=SEED,
    shuffle=True,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
)

class_names = train_ds.class_names
num_classes = len(class_names)

# Prefetching
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


print(f"Number of classes: {num_classes}")
print(f"Class names: {class_names}")
print(
    f"Training dataset size: {tf.data.experimental.cardinality(train_ds).numpy()} batches"
)
print(
    f"Validation dataset size: {tf.data.experimental.cardinality(val_ds).numpy()} batches"
)

In [ ]:
augmenter = keras.Sequential(
    [
        keras_cv.layers.RandomFlip("horizontal"),
        keras_cv.layers.RandomRotation(factor=0.1),
        keras_cv.layers.RandomZoom(height_factor=0.1, width_factor=0.1),
        keras_cv.layers.RandomContrast(factor=0.1, value_range=(0, 255)),
    ],
    name="augmenter",
)


def preprocess_data(
    images: tf.Tensor, labels: tf.Tensor
) -> tuple[tf.Tensor, tf.Tensor]:
    """Applies augmentation to images."""
    augmented_images = augmenter(images)
    return augmented_images, labels


train_ds = train_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE)

In [ ]:
# Base Model
base_model = keras.applications.MobileNetV2(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights="imagenet",
)
base_model.trainable = False

# Build the new model
inputs = keras.Input(shape=(*IMG_SIZE, 3), name="input_layer")
x = mobilenet_preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x)
x = layers.Dropout(0.3, name="dropout_layer")(x)
outputs = layers.Dense(num_classes, activation="softmax", name="output_layer")(x)

model = keras.Model(inputs, outputs, name="plant_disease_classifier")

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

In [ ]:
LOG_DIR = "logs"
Path(LOG_DIR).mkdir(parents=True, exist_ok=True)

# Callbacks
model_checkpoint_cb = ModelCheckpoint(
    filepath=Path(LOG_DIR) / "transfer_learning" / "best_model.keras",
    save_best_only=True,
    monitor="val_accuracy",
    mode="max",
    verbose=1,
)

early_stopping_cb = EarlyStopping(
    patience=5, restore_best_weights=True, monitor="val_loss", mode="min", verbose=1
)

reduce_lr_on_plateau_cb = ReduceLROnPlateau(
    patience=3, factor=0.5, verbose=1, monitor="val_loss", mode="min"
)

tensorboard_cb_tl = TensorBoard(log_dir=Path(LOG_DIR) / "transfer_learning")

transfer_learning_callbacks = [
    model_checkpoint_cb,
    early_stopping_cb,
    reduce_lr_on_plateau_cb,
    tensorboard_cb_tl,
]

In [ ]:
EPOCHS_TRANSFER_LEARNING = 20

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_TRANSFER_LEARNING,
    callbacks=transfer_learning_callbacks,
    verbose=1,
)

In [ ]:
base_model.trainable = True

print(f"Trainable variables after unfreezing: {len(model.trainable_variables)}")

# Re-compile with a lower learning rate
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

# Callbacks for Fine-tuning
model_checkpoint_cb_ft = ModelCheckpoint(
    filepath=Path(LOG_DIR) / "fine_tuning" / "best_model.keras",
    save_best_only=True,
    monitor="val_accuracy",
    mode="max",
    verbose=1,
)
tensorboard_cb_ft = TensorBoard(log_dir=Path(LOG_DIR) / "fine_tuning")

fine_tuning_callbacks = [
    model_checkpoint_cb_ft,
    early_stopping_cb,
    reduce_lr_on_plateau_cb,
    tensorboard_cb_ft,
]

In [ ]:
EPOCHS_FINE_TUNING = 10

history_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FINE_TUNING,
    callbacks=fine_tuning_callbacks,
    verbose=1,
)

In [ ]:
print("Evaluating final model on the validation set:")
loss, accuracy = model.evaluate(val_ds, verbose=0)
print(f"  Validation Loss: {loss:.4f}")
print(f"  Validation Accuracy: {accuracy * 100:.2f}%\n")

# Generate predictions for Confusion Matrix
val_labels_list = []
for _, labels_batch in val_ds:
    val_labels_list.append(labels_batch)
val_true_labels = tf.concat(val_labels_list, axis=0).numpy()

val_pred_probs = model.predict(val_ds, verbose=0)
val_pred_labels = tf.argmax(val_pred_probs, axis=1).numpy()

# Plot Confusion Matrix
cm = confusion_matrix(val_true_labels, val_pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

fig, ax = plt.subplots(figsize=(12, 12))
disp.plot(xticks_rotation=90, cmap="Blues", ax=ax, values_format="d")
plt.title("Confusion Matrix (After Fine-tuning)")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
FINAL_MODEL_PATH = "best_model.keras"
model.save(FINAL_MODEL_PATH)
print(f"Final model saved to: {FINAL_MODEL_PATH}")